# Net fusion results analysis
The notebook analyzes the results of fusing different models results in different combinations

## Libraries loading

In [1]:
import os
import ntpath
import sys
if '../' not in sys.path:
    sys.path.append('../')
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import sklearn.metrics as M
from sklearn.metrics import log_loss
from scipy.special import expit
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, train_test_split
from itertools import combinations
from isplutils.utils import aggregate
from multiprocessing import Pool, cpu_count
import seaborn as sns
from p_tqdm import p_map

## Parameters

In [2]:
results_root = Path('results/')
results_model_folder = list(results_root.glob('net-*'))
column_list = ['video', 'score', 'label']
do_distplot = False

## Helper functions

In [3]:
def compute_metrics(df_res:pd.DataFrame,train_tag:str) -> dict:
    numreal = sum(df_res['label']==False)
    numfake = sum(df_res['label']==True
)
    
    netname = train_tag.split('net-')[1].split('_')[0]
    traindb = train_tag.split('traindb-')[1].split('_')[0]
    
    loss = M.log_loss(df_res['label'],expit(df_res['score']))
    acc = M.accuracy_score(df_res['label'],df_res['score']>0)
    accbal = M.balanced_accuracy_score(df_res['label'],df_res['score']>0)
    rocauc = M.roc_auc_score(df_res['label'],df_res['score'])
    
    res_dict = {'traintag':train_tag,
                'net':netname,
                'traindb': traindb,
                'testdb':testdb,'testsplit':testsplit,
                'numreal':numreal,'numfake':numfake,
                'loss':loss,
                'acc':acc,'accbal':accbal,
                'rocauc':rocauc} 
    return res_dict

def highlight_values(s):
    '''
    highlight the three lowest and highest values in a Series (min: [red, yellow, darkorange], max: [green, lime, aqua]).
    '''
    s_idx = np.argsort(s.values)
    style = [''] * len(s)
    style[s_idx[0]] = 'background-color: red'
#     style[s_idx[1]] = 'background-color: yellow'
#     style[s_idx[2]] = 'background-color: darkorange'
    style[s_idx[-1]] = 'background-color: green'
#     style[s_idx[-2]] = 'background-color: lime'
#     style[s_idx[-3]] = 'background-color: aqua'
    return style

## Load data

In [4]:
# Load data in multi-index dataframe
if os.path.exists('data_frame_df.pkl'):
    data_frame_df = pd.read_pickle('data_frame_df.pkl')
    model_list = []
    for model_folder in tqdm(results_model_folder):
        dataset_list = []
        train_model_tag = model_folder.name
        model_results = model_folder.glob('*.pkl')
        for model_path in model_results:
            dataset_tag = os.path.splitext(ntpath.split(model_path)[1])[0]
            dataset_list.append(dataset_tag)
        model_list.append(train_model_tag)
else:
    data_model_list = []
    model_list = []
    for model_folder in tqdm(results_model_folder):
        data_dataset_list = []
        dataset_list = []
        train_model_tag = model_folder.name
        model_results = model_folder.glob('*.pkl')
        for model_path in model_results:
            netname = train_model_tag.split('net-')[1].split('_')[0]
            traindb = train_model_tag.split('traindb-')[1].split('_')[0]
            testdb, testsplit = model_path.with_suffix('').name.rsplit('_',1)
            dataset_tag = os.path.splitext(ntpath.split(model_path)[1])[0]
            df_frames = pd.read_pickle(model_path)[column_list]
            # Add info on training and test datasets
            df_frames['netname'] = netname
            df_frames['train_db'] = traindb
            df_frames['test_db'] = testdb
            df_frames['test_split'] = testsplit
            data_dataset_list.append(df_frames)
            dataset_list.append(dataset_tag)
        data_model_list.append(pd.concat(data_dataset_list, keys=dataset_list, names=['dataset']))
        model_list.append(train_model_tag)
    data_frame_df = pd.concat(data_model_list, keys=model_list, names=['model']).swaplevel(0, 1)
    data_frame_df.to_pickle('data_frame_df.pkl')

### Eliminate Xception experiments, consider only test sets

In [5]:
data_frame_df = data_frame_df[data_frame_df['test_split']=='test']
dataset_list = [x for x in dataset_list if "_val" not in x]
print('Datasets considered are {}'.format(dataset_list))
model_selection_list = ['EfficientNetB4', 'EfficientNetAutoAttB4', 'EfficientNetB4ST', 'EfficientNetAutoAttB4ST']
data_frame_df = data_frame_df[data_frame_df['netname'].isin(model_selection_list)]
model_list = data_frame_df.index.get_level_values(1).unique()
print('Models considered are {}'.format(data_frame_df['netname'].unique()))
data_frame_df

Datasets considered are ['ff-c23-720-140-140_test', 'dfdc-35-5-10_test']
Models considered are ['EfficientNetB4' 'EfficientNetB4ST' 'EfficientNetAutoAttB4'
 'EfficientNetAutoAttB4ST']


video  \
dataset                 model                                              facepath                                                             
ff-c23-720-140-140_test net-EfficientNetB4_traindb-ff-c23-720-140-140_f... manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
...                                                                                                                                       ...   
dfdc-35-5-10_test       net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-1... dfdc_train_part_47/zpujdfwivh.mp4/fr260_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr270_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr279_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr289_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr299_subj2.jpg   zpujdfwivh.mp4   

                                                                                                                                  score  \
dataset                 model                                              facepath                                                       
ff-c23-720-140-140_test net-EfficientNetB4_traindb-ff-c23-720-140-140_f... manipulated_sequences/Deepfakes/c23/videos/134_...  3.556629   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  1.067607   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  3.871128   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  3.076689   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  4.006863   
...                                                                                                                                 ...   
dfdc-35-5-10_test       net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-1... dfdc_train_part_47/zpujdfwivh.mp4/fr260_subj2.jpg   3.383530   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr270_subj2.jpg   4.214155   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr279_subj2.jpg   3.685401   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr289_subj2.jpg  -0.509255   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr299_subj2.jpg   3.152403   

                                                                                                                               label  \
dataset                 model                                              facepath                                                    
ff-c23-720-140-140_test net-EfficientNetB4_traindb-ff-c23-720-140-140_f... manipulated_sequences/Deepfakes/c23/videos/134_...   True   
                                  

## Two models

In [6]:
print(dataset_list)
index_list = ['bce_'+dataset_list[0], 'bce_'+dataset_list[1], 'rocauc_'+dataset_list[0], 'rocauc_'+dataset_list[1]] # partial index list without accuracies (not important for now)
print(index_list)

['ff-c23-720-140-140_test', 'dfdc-35-5-10_test']
['bce_ff-c23-720-140-140_test', 'bce_dfdc-35-5-10_test', 'rocauc_ff-c23-720-140-140_test', 'rocauc_dfdc-35-5-10_test']


#### Per video loss

In [7]:
results_two_df_videos = pd.DataFrame(index=index_list)

for comb in tqdm(list(combinations(np.arange(len(model_list)), 2))):
    model_comb = np.asarray(model_list)[list(comb)]
#     method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    method = '{}_+_{}'.format(model_comb[0], model_comb[1])
    results_two_df_videos[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('video')
        y_pred = expit(group_df['score'].mean().to_numpy())
        y_true = group_df['label'].prod().to_numpy()
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    results_two_df_videos.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

In [8]:
results_two_df_videos.to_csv('results_per_video_two_models_fusion.csv')
results_two_df_videos.T.style.apply(highlight_values)

,bce_ff-c23-720-140-140_test,bce_dfdc-35-5-10_test,rocauc_ff-c23-720-140-140_test,rocauc_dfdc-35-5-10_test
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.336162,0.569402,0.951224,0.904609
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.353878,0.439426,0.942449,0.929469
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.245556,1.170922,0.967895,0.722120
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.232154,1.158846,0.968342,0.726571
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.236542,1.191757,0.968036,0.728919
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.385053,0.463961,0.947105,0.923808
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.334305,0.608603,0.951237,0.886734
net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,1.172355,0.274992,0.693782,0.948574
net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.344550,0.549655,0.951792,0.895571
net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.417008,0.535327,0.939566,0.913591


#### Per frames loss

In [9]:

results_two_df_frames = pd.DataFrame(index=index_list)
    
for comb in tqdm(list(combinations(np.arange(len(model_list)), 2))):
    model_comb = np.asarray(model_list)[list(comb)]
#     method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    method = '{}_+_{}'.format(model_comb[0], model_comb[1])
    results_two_df_frames[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('facepath')
        y_pred = expit(group_df['score'].mean().to_numpy())
        y_true = group_df['label'].prod().to_numpy()
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    
    results_two_df_frames.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

In [10]:
results_two_df_frames.to_csv('results_per_frame_two_models_fusion.csv')
results_two_df_frames.T.style.apply(highlight_values)

,bce_ff-c23-720-140-140_test,bce_dfdc-35-5-10_test,rocauc_ff-c23-720-140-140_test,rocauc_dfdc-35-5-10_test
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.422755,0.712954,0.919543,0.827018
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.453748,0.616409,0.909781,0.854633
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.356611,1.314588,0.942751,0.670927
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.341059,1.305780,0.941274,0.672177
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.336959,1.351659,0.942132,0.673786
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.483896,0.641047,0.914342,0.845901
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.418181,0.750102,0.920517,0.805550
net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,1.282801,0.468704,0.669709,0.880026
net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.425044,0.693356,0.919660,0.819967
net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.487044,0.677964,0.898957,0.833053


## Three models

### Per video loss (sig(mean))

In [11]:
results_three_df_videos = pd.DataFrame(index=index_list)

for comb in tqdm(list(combinations(np.arange(len(model_list)), 3))):
    model_comb = np.asarray(model_list)[list(comb)]
#     method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    method = '{}_+_{}_+_{}'.format(model_comb[0], model_comb[1], model_comb[2])
    results_three_df_videos[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('video')
        y_pred = expit(group_df['score'].mean().to_numpy())
        y_true = group_df['label'].prod().to_numpy()
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    results_three_df_videos.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

In [12]:
results_three_df_videos.to_csv('results_per_video_three_models_fusion.csv')
results_three_df_videos.T.style.apply(highlight_values)

,bce_ff-c23-720-140-140_test,bce_dfdc-35-5-10_test,rocauc_ff-c23-720-140-140_test,rocauc_dfdc-35-5-10_test
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.486722,0.376143,0.909624,0.941975
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.275768,0.722204,0.963221,0.850394
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.282477,0.712734,0.962290,0.863701
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.288402,0.729947,0.962098,0.867066
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.510117,0.388532,0.917085,0.939866
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.461611,0.457651,0.922768,0.927848
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.280270,0.591437,0.960906,0.888649
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.285620,0.578575,0.959605,0.900540
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.291839,0.594192,0.959445,0.902918
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.532205,0.348033,0.904834,0.943802


### Per frames loss (sig(mean))

In [13]:
results_three_df_frames = pd.DataFrame(index=index_list)
    
for comb in tqdm(list(combinations(np.arange(len(model_list)), 3))):
    model_comb = np.asarray(model_list)[list(comb)]
#     method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    method = '{}_+_{}_+_{}'.format(model_comb[0], model_comb[1], model_comb[2])
    results_three_df_frames[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('facepath')
        y_pred = expit(group_df['score'].mean().to_numpy())
        y_true = group_df['label'].prod().to_numpy()
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    
    results_three_df_frames.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

In [14]:
results_three_df_frames.to_csv('results_per_frame_three_models_fusion.csv')
results_three_df_frames.T.style.apply(highlight_values)

,bce_ff-c23-720-140-140_test,bce_dfdc-35-5-10_test,rocauc_ff-c23-720-140-140_test,rocauc_dfdc-35-5-10_test
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.574290,0.541452,0.872719,0.872025
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.360952,0.856276,0.937611,0.779356
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.363668,0.848757,0.933787,0.787233
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.363431,0.874176,0.934686,0.789856
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.594767,0.551618,0.879830,0.868714
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.535814,0.598160,0.886623,0.853121
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.372252,0.750173,0.934849,0.814396
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.373696,0.739164,0.930663,0.822618
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.374828,0.765886,0.931047,0.824165
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.628700,0.529569,0.868083,0.873516


## Four models

### Per video loss (sig(mean))

In [15]:
results_four_df_videos = pd.DataFrame(index=index_list)

for comb in tqdm(list(combinations(np.arange(len(model_list)), 4))):
    model_comb = np.asarray(model_list)[list(comb)]
#     method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    method = '{}_+_{}_+_{}_+_{}'.format(model_comb[0], model_comb[1], model_comb[2], model_comb[3])
    results_four_df_videos[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('video')
        y_pred = expit(group_df['score'].mean().to_numpy())
        y_true = group_df['label'].prod().to_numpy()
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    results_four_df_videos.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

In [16]:
results_four_df_videos.to_csv('results_per_video_four_models_fusion.csv')
results_four_df_videos.T.style.apply(highlight_values)

,bce_ff-c23-720-140-140_test,bce_dfdc-35-5-10_test,rocauc_ff-c23-720-140-140_test,rocauc_dfdc-35-5-10_test
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.341360,0.478467,0.949649,0.920497
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.367352,0.471958,0.944031,0.927152
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.375174,0.482126,0.943304,0.929054
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.631637,0.331521,0.876059,0.946901
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.583628,0.360510,0.882404,0.943066
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.259068,0.803351,0.965963,0.826901
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.263887,0.819117,0.965147,0.827657
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.358511,0.492280,0.951186,0.916963
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.334481,0.570957,0.953278,0.896061
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.267190,0.811112,0.965816,0.838792


### Per frames loss (sig(mean))

In [17]:
results_four_df_frames = pd.DataFrame(index=index_list)
    
for comb in tqdm(list(combinations(np.arange(len(model_list)), 4))):
    model_comb = np.asarray(model_list)[list(comb)]
#     method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    method = '{}_+_{}_+_{}_+_{}'.format(model_comb[0], model_comb[1], model_comb[2], model_comb[3])
    results_four_df_frames[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('facepath')
        y_pred = expit(group_df['score'].mean().to_numpy())
        y_true = group_df['label'].prod().to_numpy()
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    
    results_four_df_frames.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

In [18]:
results_four_df_frames.to_csv('results_per_frame_four_models_fusion.csv')
results_four_df_frames.T.style.apply(highlight_values)

,bce_ff-c23-720-140-140_test,bce_dfdc-35-5-10_test,rocauc_ff-c23-720-140-140_test,rocauc_dfdc-35-5-10_test
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.421441,0.628671,0.920662,0.849270
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.444147,0.623940,0.912116,0.853830
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.448448,0.641582,0.911530,0.855206
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.721146,0.504569,0.837896,0.878935
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.664155,0.519142,0.844317,0.873615
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.341720,0.933912,0.941033,0.758207
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.344234,0.958793,0.940567,0.757808
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.437937,0.641223,0.922408,0.844602
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval,0.406985,0.701928,0.925334,0.822353
net-EfficientNetB4_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_bestval_+_net-EfficientNetB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval_+_net-EfficientNetAutoAttB4ST_traindb-ff-c23-720-140-140_face-scale_size-224_seed-41_bestval,0.342684,0.949269,0.939219,0.765705


## Evaluate all combinations

In [19]:
def get_best_loss_weights(y_true, y_pred):
    num_th = 20
    a_list = np.linspace(0, 1, num_th)
    b_list = np.linspace(0, 1, num_th)
    loss_list = []
    aa_list = []
    bb_list = []
    for a in a_list:
        for b in b_list:
            if a + b < 1:
                loss_list.append(log_loss( y_true, expit(y_pred.T.dot([a, b, 1-a-b])) ))
                aa_list.append(a)
                bb_list.append(b)
    best_idx = np.argmin(loss_list)
    return loss_list[best_idx], aa_list[best_idx], bb_list[best_idx]

def get_best_auc_weights(y_true, y_pred):
    num_th = 20
    a_list = np.linspace(0, 1, num_th)
    b_list = np.linspace(0, 1, num_th)
    loss_list = []
    aa_list = []
    bb_list = []
    for a in a_list:
        for b in b_list:
            if a + b < 1:
                loss_list.append(M.roc_auc_score( y_true, expit(y_pred.T.dot([a, b, 1-a-b])) ))
                aa_list.append(a)
                bb_list.append(b)
    best_idx = np.argmin(loss_list)
    return loss_list[best_idx], aa_list[best_idx], bb_list[best_idx]

In [ ]:
# Aggregate
comb_list_1 = list(combinations(np.arange(len(model_list)), 1))
comb_list_2 = list(combinations(np.arange(len(model_list)), 2))
comb_list_3 = list(combinations(np.arange(len(model_list)), 3))
comb_list_4 = list(combinations(np.arange(len(model_list)), 4))
comb_list = comb_list_1 + comb_list_2 + comb_list_3 + comb_list_4
loss_list = dict()
loss_list['bce'] = []
loss_list['auc'] = []
model_loss_comb_list = []
a_loss_list = []
b_loss_list = []
model_auc_comb_list = []
a_auc_list = []
b_auc_list = []
for comb in tqdm(comb_list):
    model_comb = np.asarray(model_list)[list(comb)]
    y_pred = []
    y_true = []
    for dataset in dataset_list:
        group_df = data_frame_df.loc[dataset].loc[model].groupby('video')
        y_pred.append(group_df['score'].mean().to_numpy())
        y_true.append(group_df['label'].prod().to_numpy())
    y_true = np.array(y_true).mean(axis=0)
    y_pred = np.array(y_pred)
    loss, a, b = get_best_loss_weights(y_true, y_pred)
    loss_list.append(loss)
    a_loss_list.append(a)
    b_loss_list.append(b)
    model_loss_comb_list.append(model_comb)
    loss, a, b = get_best_auc_weights(y_true, y_pred)
    auc_list.append(loss)
    a_auc_list.append(a)
    b_auc_list.append(b)
    model_auc_comb_list.append(model_comb)